# Notebook 7: Entrenamiento con métricas completas y emisiones

En este notebook entrenamos modelos SVD sobre los datasets reducidos (aleatorios e informados), calculamos métricas completas de recomendación y registramos las emisiones de carbono. 
Los resultados se guardan en un único archivo: **recommendation_metrics.csv**.

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from codecarbon import EmissionsTracker
from collections import defaultdict

# Configuración de rutas

In [2]:
data_path = Path("../data/processed")
subsampled_random_path = data_path / "subsampled_random"
subsampled_informed_path = data_path / "subsampled_informed"

results_path = Path("../results")
results_path.mkdir(exist_ok=True, parents=True)

output_file = results_path / "recommendation_metrics.csv"

# Funciones auxiliares para métricas
Incluyen Precision@K, Recall@K, Coverage, Diversity y Novelty.

In [3]:
def get_top_k(predictions, k=10):
    user_ratings = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        user_ratings[uid].append((iid, est))
    for uid, ratings in user_ratings.items():
        ratings.sort(key=lambda x: x[1], reverse=True)
        user_ratings[uid] = [iid for (iid, _) in ratings[:k]]
    return user_ratings


def precision_recall_at_k(predictions, k=10, threshold=3.5):
    top_k = get_top_k(predictions, k)
    user_metrics = []

    for uid, user_predictions in top_k.items():
        true_items = [iid for (u, iid, true, _, _) in predictions if u == uid and true >= threshold]

        if len(user_predictions) == 0:
            continue

        precision = len(set(user_predictions) & set(true_items)) / k if k > 0 else 0
        recall = len(set(user_predictions) & set(true_items)) / len(true_items) if len(true_items) > 0 else 0

        user_metrics.append((precision, recall))

    if not user_metrics:
        return 0, 0

    precision_mean = np.mean([x[0] for x in user_metrics])
    recall_mean = np.mean([x[1] for x in user_metrics])

    return precision_mean, recall_mean


def coverage(predictions):
    items_pred = set([iid for (_, iid, _, _, _) in predictions])
    return len(items_pred)


def diversity(predictions):
    users = list({uid for (uid, _, _, _, _) in predictions})
    overlaps = []
    for u1 in users:
        items1 = {iid for (u, iid, _, _, _) in predictions if u == u1}
        for u2 in users:
            if u1 >= u2:
                continue
            items2 = {iid for (u, iid, _, _, _) in predictions if u == u2}
            overlap = len(items1 & items2)
            overlaps.append(overlap)
    return np.mean(overlaps) if overlaps else 0


def novelty(predictions):
    item_counts = defaultdict(int)
    for (_, iid, _, _, _) in predictions:
        item_counts[iid] += 1
    return np.mean(list(item_counts.values()))

# Función de entrenamiento completa (incluye emisiones + métricas)

In [4]:
def train_and_evaluate(file_path, label):
    print(f"  -> Cargando datos desde: {file_path.name}")
    df = pd.read_csv(file_path)

    reader = Reader(rating_scale=(df['rating'].min(), df['rating'].max()))
    data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

    trainset, testset = train_test_split(data, test_size=0.2)

    tracker = EmissionsTracker()
    tracker.start()

    algo = SVD()
    print(f"  -> Entrenando modelo SVD...")
    algo.fit(trainset)
    predictions = algo.test(testset)

    emissions = tracker.stop()

    # RMSE y MAE
    rmse_values = [abs(true - est) ** 2 for (_, _, true, est, _) in predictions]
    mae_values = [abs(true - est) for (_, _, true, est, _) in predictions]
    rmse = np.sqrt(np.mean(rmse_values))
    mae = np.mean(mae_values)

    # Métricas top-K
    precision, recall = precision_recall_at_k(predictions)
    cov = coverage(predictions)

    # Div y Novelty desactivados para acelerar
    div = 0
    nov = 0

    print(f"  -> RMSE: {rmse:.4f}, MAE: {mae:.4f}, Precision@K: {precision:.4f}, Recall@K: {recall:.4f}, Coverage: {cov}")

    return {
        "dataset": label,
        "rmse": rmse,
        "mae": mae,
        "precision_at_k": precision,
        "recall_at_k": recall,
        "coverage": cov,
        "diversity": div,
        "novelty": nov,
        "emissions_kg": emissions
    }


# Entrenamiento sobre los datasets aleatorios e informados

In [5]:
fractions = [10, 25, 50, 75]
results = []

# Aleatorios
for frac in fractions:
    file_path = subsampled_random_path / f"ratings_{frac}.csv"
    print(f"Entrenando aleatorio {frac}%...")
    metrics = train_and_evaluate(file_path, f"random_{frac}")
    results.append(metrics)

# Informados usuarios
for frac in fractions:
    file_path = subsampled_informed_path / f"ratings_top_users_{frac}.csv"
    print(f"Entrenando informado usuarios {frac}%...")
    metrics = train_and_evaluate(file_path, f"informed_users_{frac}")
    results.append(metrics)

# Informados items
for frac in fractions:
    file_path = subsampled_informed_path / f"ratings_top_items_{frac}.csv"
    print(f"Entrenando informado items {frac}%...")
    metrics = train_and_evaluate(file_path, f"informed_items_{frac}")
    results.append(metrics)

Entrenando aleatorio 10%...
  -> Cargando datos desde: ratings_10.csv


[codecarbon WARNING @ 18:39:44] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 18:39:44] [setup] RAM Tracking...
[codecarbon INFO @ 18:39:44] [setup] CPU Tracking...
[codecarbon WARNING @ 18:39:46] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon INFO @ 18:39:46] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-6400 CPU @ 2.70GHz
[codecarbon WARNING @ 18:39:46] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 18:39:46] [setup] GPU Tracking...
[codecarbon INFO @ 18:39:46] No GPU found.
[codecarbon INFO @ 18:39:46] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 18:39:46] >>> Tracker's metadata:
[cod

  -> Entrenando modelo SVD...


[codecarbon INFO @ 18:39:47] Energy consumed for RAM : 0.000004 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 18:39:47] Delta energy consumed for CPU with constant : 0.000047 kWh, power : 130.0 W
[codecarbon INFO @ 18:39:47] Energy consumed for All CPU : 0.000047 kWh
[codecarbon INFO @ 18:39:47] 0.000051 kWh of electricity and 0.000000 L of water were used since the beginning.


  -> RMSE: 0.9632, MAE: 0.7641, Precision@K: 0.2283, Recall@K: 0.8353, Coverage: 2690
Entrenando aleatorio 25%...
  -> Cargando datos desde: ratings_25.csv


...

  -> RMSE: 0.8692, MAE: 0.6810, Precision@K: 0.6742, Recall@K: 0.6467, Coverage: 1853
Entrenando informado items 75%...
  -> Cargando datos desde: ratings_top_items_75.csv


[codecarbon WARNING @ 19:03:32] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 19:03:32] [setup] RAM Tracking...
[codecarbon INFO @ 19:03:32] [setup] CPU Tracking...
[codecarbon WARNING @ 19:03:34] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon INFO @ 19:03:34] CPU Model on constant consumption mode: Intel(R) Core(TM) i5-6400 CPU @ 2.70GHz
[codecarbon WARNING @ 19:03:34] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 19:03:34] [setup] GPU Tracking...
[codecarbon INFO @ 19:03:34] No GPU found.
[codecarbon INFO @ 19:03:34] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 19:03:34] >>> Tracker's metadata:
[cod

  -> Entrenando modelo SVD...


[codecarbon INFO @ 19:03:47] Energy consumed for RAM : 0.000035 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 19:03:47] Delta energy consumed for CPU with constant : 0.000456 kWh, power : 130.0 W
[codecarbon INFO @ 19:03:47] Energy consumed for All CPU : 0.000456 kWh
[codecarbon INFO @ 19:03:47] 0.000491 kWh of electricity and 0.000000 L of water were used since the beginning.


  -> RMSE: 0.8691, MAE: 0.6821, Precision@K: 0.6818, Recall@K: 0.6355, Coverage: 2779


# Guardado de resultados en CSV único

In [6]:
df_results = pd.DataFrame(results)
df_results.to_csv(output_file, index=False)
df_results

,dataset,rmse,mae,precision_at_k,recall_at_k,coverage,diversity,novelty,emissions_kg
0,random_10,0.963246,0.764052,0.228265,0.835315,2690,0,0,0.000009
1,random_25,0.941447,0.745772,0.378964,0.839127,3107,0,0,0.000023
2,random_50,0.918194,0.724797,0.522936,0.769830,3320,0,0,0.000040
3,random_75,0.892747,0.702913,0.618779,0.694492,3403,0,0,0.000072
4,informed_users_10,0.861041,0.676441,0.854801,0.161023,3309,0,0,0.000036
5,informed_users_25,0.854523,0.671141,0.854371,0.231428,3394,0,0,0.000053
6,informed_users_50,0.862720,0.677257,0.830066,0.365951,3456,0,0,0.000067
7,informed_users_75,0.870710,0.683350,0.778079,0.522523,3466,0,0,0.000078
8,informed_items_10,0.861022,0.674104,0.584657,0.774081,370,0,0,0.000034
9,informed_items_25,0.861763,0.674610,0.649170,0.686108,926,0,0,0.000054


 ##### El archivo `recommendation_metrics.csv` está listo para ser analizado.